In [315]:
import numpy as np

In [316]:
def forward_layer_matrixMult(x, w):
    # both w,x are row vectors
    # x : n * d, w : d * 1
    # N = xw

    return x @ w

In [317]:
def backward_layer_matrixMult(x, w):
    # N = xw
    # dN/dw, dN/dx

    return x, w

In [318]:
def forward_layer_biasAdd(N, b):
    # N : n * 1, b : 1 * 1
    # P = N + b

    # N: n * k, b: k * 1
     
    return N + b

In [319]:
def backward_layer_biasAdd(N, b):
    # P = N + b (n * 1)
    # dP/dN = Identity (n * n)
    # dP/db = 1 (n * 1)

    #return np.identity(N.shape[0]), np.ones((N.shape[0], 1))
    return np.identity(N.shape[0]), np.identity(b.shape[0])

In [320]:
def forward_layer_sigmoid(P):
    # Q = 1 / 1 + e^-P_i
    
    return 1 / (1 + np.exp(-P))

In [321]:
def backward_layer_sigmoid(P):
    # dQ_i/dP_i = Q_i * (1 - Q_i)

    Q = forward_layer_sigmoid(P)
    Q1 = Q * (1 - Q)
    
    return np.diag(Q1)

In [322]:
def forward_layer_softMax(Q):
    denom = np.sum(np.exp(Q))
    return np.exp(Q) / denom

In [323]:
def backward_layer_softMax(P):
    # dQ/dP = formula written in copy

    sum_ek = np.sum(np.exp(P))
    n = P.shape[0]

    dQ = np.zeros(n, n)

    for i in range(n):
        for j in range(n):
            if i == j :
                dQ[i][j] = sum_ek
            
            dQ[i][j] -= np.exp(P[i])
            dQ[i][j] *= np.exp(P[j])
            dQ[i][j] /= sum_ek * sum_ek

    return dQ

In [324]:
def forward_layer_meanSqrLoss(P,y):
    # P : n * 1, y : n * 1
    # L = 1/n * sum((P - y) ^ 2)    : (1 * 1)
    
    return np.sum((P - y)**2)
    # k = P - y
    # res = k.T @ k

    # return res / 2

In [325]:
def backward_layer_meanSqrLoss(P, y):
    # L = 1/n * sum((P - y) ^ 2)    : (1 * 1)
    # dL/dP = 2 * (P - y) : (n * 1)

    return 2 * (P - y)

In [326]:
def forward_layer_crossEntropy(P,y):
    #y can be multiclass or 
    #P,y both are matrix of same shape
        # returns loss_vector

    loss = np.log((P ** y))
    return -np.sum(loss) 

In [327]:
def backward_layer_crossEntropy(Q,y):
    # dl/dp[i][j] = Q[i][j] - y[i][j]
    return Q - y

### Q2 Boston Dataset

In [328]:
import pandas as pd
import numpy as np
import sklearn.datasets as sk

In [329]:
dictt = sk.fetch_california_housing()
data = dictt.data
target = dictt.target
data.shape

(20640, 8)

In [330]:
from sklearn.preprocessing import Normalizer

In [331]:
#Normalising dataset

transformer = Normalizer().fit(data)

# mean_vals = np.mean(data, axis=0)
# std_devs = np.std(data, axis=0)
# epsilon = 1e-8
# data = (data - mean_vals) / (std_devs + epsilon)

In [332]:
data = transformer.transform(data)

In [333]:
valid = 20000
test_x = data[valid+1:]
test_y = target[valid+1:]
x = data[0:valid]
y = target[0:valid]

In [334]:
def forwardProp(x,w,b,y):
    N = forward_layer_matrixMult(x,w)
    P = forward_layer_biasAdd(N,b)
    L = forward_layer_meanSqrLoss(P,y)
    print("Error:", L/y.shape[0])
    return N,P

In [335]:
def backwardProp(x,w,b,N,P,y):
    dL_dP = backward_layer_meanSqrLoss(P,y)
    dP_dN,dP_dB = backward_layer_biasAdd(N,b)
    dN_dw,dN_dx = backward_layer_matrixMult(x,w)
    dl_dw = dN_dw.T @ dP_dN @ dL_dP
    dl_db =  dP_dB.T @ dL_dP
    return dl_dw,dl_db

In [336]:
def stochastic_gd(x,w,b,y,alpha,epoch):
    for j in range(epoch):
        i = np.random.randint(y.shape[0])
        x_ar = x[i].reshape(1,-1)
        y_ar = np.array([y[i]])
        N,P = forwardProp(x_ar,w,b,y_ar)
        del_w=x_ar.T @ (2*(y_ar-P))
        del_b=2*(y[i]-P)
        #del_w,del_b = backwardProp(x_ar,w,b,N,P,y_ar)
        # print("del w, del b : ", del_w, del_b)
        w = w - (del_w * alpha)
        b = b - (del_b * alpha)
    return w, b

In [337]:
w = np.random.rand(len(data[0]),1)
b = np.random.rand(1,1)
alpha = 1e-8
epoch = 50000
w_ = w

In [338]:
w,b = stochastic_gd(x,w,b,y,alpha,epoch)

Error: 0.11669750285972416
Error: 0.3682014930044176
Error: 0.018747647707148894
Error: 6.108553205071474
Error: 0.06077263303587706
Error: 0.21518281253616142
Error: 0.1735050238307117
Error: 4.714378360541469
Error: 12.274563540870979
Error: 0.34183892170208363
Error: 0.048886521072193004
Error: 1.2527379811102728
Error: 0.42218499311317137
Error: 0.052123732252480084
Error: 0.02416418386163768
Error: 0.6968827564076947
Error: 0.07730576327240543
Error: 0.11834971578472121
Error: 1.0856905103124148
Error: 1.7638090300636302
Error: 0.052232333871267474
Error: 0.1818150801718072
Error: 5.907222307035421
Error: 3.382086792406674
Error: 0.0003113186762544787
Error: 0.07252510850986232
Error: 0.11522354194648682
Error: 0.011779803622122295
Error: 0.1578585126426602
Error: 2.6779017386939947
Error: 0.734043653743087
Error: 6.634629142146842
Error: 5.302357957691802
Error: 0.004915026154425833
Error: 0.09096504836207847
Error: 0.011818636215749969
Error: 0.18752454656946052
Error: 12.187796

In [339]:
w

array([[0.69553164],
       [0.54659375],
       [0.62856507],
       [0.53143643],
       [0.84095556],
       [0.1924119 ],
       [0.32182412],
       [0.08709617]])

In [340]:
N,P = forwardProp(test_x,w,b,test_y)

Error: 863.3660797836337


### Q3 IRIS

In [341]:
d = sk.load_iris()
x = d.data
y = d.target

In [342]:
x.shape

(150, 4)

In [343]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [344]:
def oneHot_encoding(categories ,labels):
    num_samples = len(labels)
    num_categories = len(categories)
    one_hot = np.zeros((num_samples, num_categories))

    # Perform one-hot encoding
    for i, label in enumerate(labels):
        index = categories.index(label)
        one_hot[i, index] = 1
    return one_hot

In [345]:
y_encoded = oneHot_encoding([0,1,2],y)
y_encoded

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [346]:
def forwardProp(x,w,b,y):
    N = forward_layer_matrixMult(x,w)
    P = forward_layer_biasAdd(N,b)
    Q = forward_layer_softMax(P)
    L = forward_layer_crossEntropy(Q,y)
    print("Error:", L)
    return Q

In [347]:
w = np.random.rand(x.shape[1],y_encoded.shape[1])
b = np.random.rand(1,y_encoded.shape[1])
w

array([[0.6552062 , 0.44243824, 0.50566856],
       [0.43876021, 0.67293217, 0.61630479],
       [0.4002245 , 0.31340213, 0.75771848],
       [0.87238397, 0.30629434, 0.29170385]])

In [348]:
def backwardProp(x, Q, y):
    dL_dP = backward_layer_crossEntropy(Q, y)
    dL_dw = x.T @ dL_dP
    dL_db = np.sum(Q-y,axis = 0)
    return dL_dw/1000, dL_db/1000

In [349]:
def gd(x,w,b,y,eta):
    Q = forwardProp(x,w,b,y)
    dL_dw,dL_db = backwardProp(x,Q,y)
    w = w - (eta * dL_dw)
    b = b - (eta * dL_db)
    return w,b

In [350]:
epoch =600
eta = 1e-3
for i in range(epoch):
    w,b = gd(x,w,b,y_encoded,eta)

Error: 1154.3584718670452
Error: 1154.449328823615
Error: 1154.5403295089484
Error: 1154.6314738959675
Error: 1154.7227619573932
Error: 1154.8141936657457
Error: 1154.9057689933443
Error: 1154.997487912308
Error: 1155.0893503945563
Error: 1155.1813564118077
Error: 1155.2735059355819
Error: 1155.3657989371982
Error: 1155.4582353877774
Error: 1155.5508152582406
Error: 1155.6435385193104
Error: 1155.7364051415102
Error: 1155.8294150951656
Error: 1155.922568350404
Error: 1156.0158648771535
Error: 1156.1093046451463
Error: 1156.2028876239156
Error: 1156.2966137827975
Error: 1156.3904830909319
Error: 1156.4844955172607
Error: 1156.5786510305293
Error: 1156.6729495992872
Error: 1156.7673911918878
Error: 1156.8619757764873
Error: 1156.9567033210474
Error: 1157.0515737933342
Error: 1157.146587160917
Error: 1157.2417433911726
Error: 1157.3370424512805
Error: 1157.4324843082277
Error: 1157.528068928805
Error: 1157.6237962796104
Error: 1157.7196663270474
Error: 1157.8156790373264
Error: 1157.91183

In [351]:
w

array([[0.80420847, 0.62036457, 0.70043798],
       [0.54106974, 0.75596105, 0.70424056],
       [0.443112  , 0.44107914, 0.92188847],
       [0.87942085, 0.34603159, 0.3516579 ]])

In [352]:
def predict(x,y_encoded,y,w,b):
    Q = forwardProp(x,w,b,y_encoded)
    return 1-np.count_nonzero(y - np.argmax(Q,axis =1))/y.shape[0]

In [353]:
predict(x,y_encoded,y,w,b)

Error: 1232.8419219904242


0.33333333333333337